In [45]:
import pandas as pd


In [46]:
data = pd.read_csv('./data/sample-data.csv', header=None, on_bad_lines='skip')

data.columns = column_names = [
"address",
"price",
"gross_tax",
"strata_maintenance_fees",
"bedrooms",
"bathrooms",
"property_type",
"property_age",
"title",
"style",
"heating_type",
"feature",
"amenities",
"appliances",
"community",
"days_on_rew",
"property_views",
"mls®_number",
"source",
'frontage', 
'lot_size', 
'year_built', 
'depth']
data.head()


,address,price,gross_tax,strata_maintenance_fees,bedrooms,bathrooms,property_type,property_age,title,style,...,appliances,community,days_on_rew,property_views,mls®_number,source,frontage,lot_size,year_built,depth
0,3735 Puget Drive,"$5,498,800","$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122
1,3735 Puget Drive,"$5,498,800","$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122
2,302 5118 Cambie Street,"$2,367,900",$0,$925,3,3,apt/condo,NaN,freehold strata,1 storey,...,"in suite laundry, microwave, range top, washer...",cambie,109 days,592,r2712600,rebgv,NaN,NaN,built in 2022 (0 yrs old),NaN
3,438 W 17th Avenue,"$3,888,000","$10,064",NaN,8,4,multifamily,NaN,freehold nonstrata,NaN,...,NaN,cambie,61 days,1129,r2722725,rebgv,49.70 feet,49 ft x 137 ft (6807 ft²),built in 1930 (92 yrs old),137.5
4,2505 1151 W Georgia Street,"$2,470,000","$5,658","$1,215",2,2,apt/condo,NaN,freehold strata,upper unit,...,in suite laundry,coal harbour,57 days,789,r2724260,rebgv,NaN,NaN,built in 2016 (6 yrs old),NaN


In [47]:

# houses.drop(houses.columns[15:], axis=1, inplace=True)
# house_columns = ["address",
# "price",
# "gross_tax",
# "stratahouses = data[data[6] == 'House']_maintenance_fees",
# "bedrooms",
# "bathrooms",
# "property_type",
# "property_age",
# "title",
# "style",
# "heating_type",
# "feature",
# "amenities",
# "appliances",
# "community",
# "days_on_rew",
# "property_views",
# "mls®_number",
# "source",
# 'frontage', 
# 'lot_size', 
# 'year_built', 
# 'depth']
# houses.columns = house_columns
# houses.head()

In [48]:
# Filter Houses Only
houses = data[data['property_type'] == 'house']
houses.head()

,address,price,gross_tax,strata_maintenance_fees,bedrooms,bathrooms,property_type,property_age,title,style,...,appliances,community,days_on_rew,property_views,mls®_number,source,frontage,lot_size,year_built,depth
0,3735 Puget Drive,"$5,498,800","$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122
1,3735 Puget Drive,"$5,498,800","$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122
7,3735 Puget Drive,"$5,498,800","$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122
8,3735 Puget Drive,"$5,498,800","$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122
10,3735 Puget Drive,"$5,498,800","$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122


In [49]:
# Convert Price column to integer
houses['price'] = houses['price'].astype(str)
houses['price'] = houses['price'].str.replace(',', '')
houses['price'] = houses['price'].str.replace('$', '')
houses['price'] = houses['price'].astype(int)
houses.head()

/var/folders/hx/wsc86f0s71j8zp38fr7x77pc0000gn/T/ipykernel_6903/636961071.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['price'] = houses['price'].astype(str)
/var/folders/hx/wsc86f0s71j8zp38fr7x77pc0000gn/T/ipykernel_6903/636961071.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['price'] = houses['price'].str.replace(',', '')
/var/folders/hx/wsc86f0s71j8zp38fr7x77pc0000gn/T/ipykernel_6903/636961071.py:4: FutureWarning: The default value of regex will change from True to False in a f

,address,price,gross_tax,strata_maintenance_fees,bedrooms,bathrooms,property_type,property_age,title,style,...,appliances,community,days_on_rew,property_views,mls®_number,source,frontage,lot_size,year_built,depth
0,3735 Puget Drive,5498800,"$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122
1,3735 Puget Drive,5498800,"$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122
7,3735 Puget Drive,5498800,"$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122
8,3735 Puget Drive,5498800,"$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122
10,3735 Puget Drive,5498800,"$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122


In [50]:
# convert Lot Size to Integer
houses['lot_size'] = houses['lot_size'].astype(str)
houses['lot_size'] = houses['lot_size'].map(lambda x: x[x.rfind("(") + 1:x.rfind("ft²") -1])
houses['lot_size'] = houses['lot_size'].astype(int)
houses.head()


/var/folders/hx/wsc86f0s71j8zp38fr7x77pc0000gn/T/ipykernel_6903/2123309634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['lot_size'] = houses['lot_size'].astype(str)
/var/folders/hx/wsc86f0s71j8zp38fr7x77pc0000gn/T/ipykernel_6903/2123309634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['lot_size'] = houses['lot_size'].map(lambda x: x[x.rfind("(") + 1:x.rfind("ft²") -1])
/var/folders/hx/wsc86f0s71j8zp38fr7x77pc0000gn/T/ipykernel_6903/2123309634.py:4: SettingWithCopyWarning: 
A value

,address,price,gross_tax,strata_maintenance_fees,bedrooms,bathrooms,property_type,property_age,title,style,...,appliances,community,days_on_rew,property_views,mls®_number,source,frontage,lot_size,year_built,depth
0,3735 Puget Drive,5498800,"$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,6954,built in 1987 (35 yrs old),122
1,3735 Puget Drive,5498800,"$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,6954,built in 1987 (35 yrs old),122
7,3735 Puget Drive,5498800,"$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,6954,built in 1987 (35 yrs old),122
8,3735 Puget Drive,5498800,"$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,6954,built in 1987 (35 yrs old),122
10,3735 Puget Drive,5498800,"$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,6954,built in 1987 (35 yrs old),122


In [51]:
# Convert Gross Tax Income to Integer
houses['gross_tax'] = houses['gross_tax'].astype(str)
houses['gross_tax'] = houses['gross_tax'].str.replace(',', '')
houses['gross_tax'] = houses['gross_tax'].str.replace('$', '')
houses['gross_tax'] = houses['gross_tax'].astype(int)
houses.head()

/var/folders/hx/wsc86f0s71j8zp38fr7x77pc0000gn/T/ipykernel_6903/2046443693.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['gross_tax'] = houses['gross_tax'].astype(str)
/var/folders/hx/wsc86f0s71j8zp38fr7x77pc0000gn/T/ipykernel_6903/2046443693.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['gross_tax'] = houses['gross_tax'].str.replace(',', '')
/var/folders/hx/wsc86f0s71j8zp38fr7x77pc0000gn/T/ipykernel_6903/2046443693.py:4: FutureWarning: The default value of regex will change from T

,address,price,gross_tax,strata_maintenance_fees,bedrooms,bathrooms,property_type,property_age,title,style,...,appliances,community,days_on_rew,property_views,mls®_number,source,frontage,lot_size,year_built,depth
0,3735 Puget Drive,5498800,19041,NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,6954,built in 1987 (35 yrs old),122
1,3735 Puget Drive,5498800,19041,NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,6954,built in 1987 (35 yrs old),122
7,3735 Puget Drive,5498800,19041,NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,6954,built in 1987 (35 yrs old),122
8,3735 Puget Drive,5498800,19041,NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,6954,built in 1987 (35 yrs old),122
10,3735 Puget Drive,5498800,19041,NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,6954,built in 1987 (35 yrs old),122


In [52]:
# Convert House Age to integer - Consider prebuild houses with age 0
houses['year_built'] = houses['year_built'].astype(str)
houses['year_built'] = houses['year_built'].map(lambda x: x[x.rfind("(") + 1:x.rfind("yrs") -1])
houses['year_built'] = houses['year_built'].astype(int)

## remove duplicate based on mls number
houses = houses.drop_duplicates(subset=['mls®_number'])
houses.head()

/var/folders/hx/wsc86f0s71j8zp38fr7x77pc0000gn/T/ipykernel_6903/4038970491.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['year_built'] = houses['year_built'].astype(str)
/var/folders/hx/wsc86f0s71j8zp38fr7x77pc0000gn/T/ipykernel_6903/4038970491.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['year_built'] = houses['year_built'].map(lambda x: x[x.rfind("(") + 1:x.rfind("yrs") -1])
/var/folders/hx/wsc86f0s71j8zp38fr7x77pc0000gn/T/ipykernel_6903/4038970491.py:4: SettingWithCopyWarning: 

,address,price,gross_tax,strata_maintenance_fees,bedrooms,bathrooms,property_type,property_age,title,style,...,appliances,community,days_on_rew,property_views,mls®_number,source,frontage,lot_size,year_built,depth
0,3735 Puget Drive,5498800,19041,NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,6954,35,122
22,2127 W 21st Avenue,4360000,15988,NaN,7,5,house,NaN,freehold nonstrata,3 storey,...,NaN,arbutus,88 days,601,r2689450,rebgv,50.00 feet,6100,33,122
23,2268 W 19th Avenue,6880000,23147,NaN,5,8,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,washer/dryer/fridge/stove/dishwasher,arbutus,88 days,540,r2682157,rebgv,50.00 feet,6104,5,122
24,3894 Quesnel Drive,5888000,12189,NaN,3,4,house,NaN,freehold nonstrata,3 storey,...,washer/dryer/fridge/stove/dishwasher,arbutus,88 days,284,r2654152,rebgv,44.00 feet,4100,89,93.2
26,2690 W 21st Avenue,4299000,14265,NaN,7,6,house,NaN,freehold nonstrata,"2 storey w/bsmt.,corner unit",...,"disposal - waste, in suite laundry, washer/dry...",arbutus,231 days,1440,r2669524,rebgv,60.00 feet,7320,28,122


In [53]:

houses.to_csv('data/afterCleaning.csv')